# LLM Response Metrics - Paper Tables

In [1]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, Markdown

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.precision", 1)

In [2]:
metrics_path = Path("metrics_summary.jsonl")

metrics = []
with open(metrics_path, "r") as f:
    for line in f:
        metrics.append(json.loads(line))

df = pd.DataFrame(metrics)
print(f"Total records: {len(df)}")

df = df[df["valid_for_metrics"] == True].copy()
print(f"Valid records: {len(df)}")

Total records: 1039
Valid records: 980


In [3]:
MODEL_DISPLAY_NAMES = {
    "Qwen_Qwen3-4B-Thinking-2507": "Qwen3-4B-Thinking",
    "Qwen_Qwen3-4B": "Qwen3-4B",
    "Qwen_Qwen3-14B": "Qwen3-14B",
    "Qwen_Qwen3-32B": "Qwen3-32B",
    "deepseek-ai_DeepSeek-R1-Distill-Qwen-7B": "DeepSeek-R1-7B",
    "microsoft_phi-4-reasoning": "Phi-4-reasoning",
    "microsoft_phi-4-reasoning-plus": "Phi-4-reasoning-plus",
}

MODEL_ORDER = [
    "Qwen_Qwen3-4B-Thinking-2507",  # Qwen3-4B-Thinking
    "Qwen_Qwen3-4B",  # Qwen3-4B
    "Qwen_Qwen3-14B",  # Qwen3-14B
    "deepseek-ai_DeepSeek-R1-Distill-Qwen-7B",  # DS-R1-7B
    "Qwen_Qwen3-32B",  # Qwen3-32B
    "microsoft_phi-4-reasoning",  # Phi-4-reasoning
    "microsoft_phi-4-reasoning-plus",  # Phi-4-reasoning
]

MODEL_THRESHOLDS = {
    "Qwen_Qwen3-4B-Thinking-2507": 0.25,
    "Qwen_Qwen3-4B": 0.25,
    "Qwen_Qwen3-14B": 0.25,
    "Qwen_Qwen3-32B": 0.25,
    "deepseek-ai_DeepSeek-R1-Distill-Qwen-7B": 0.1,
    "microsoft_phi-4-reasoning": 1.0,
    "microsoft_phi-4-reasoning-plus": 1.0,
}
DEFAULT_THRESHOLD = 0.25

METHOD_DISPLAY = {
    "rollout": "(baseline)",
    "nowait": "+NoWait",
    "thinkless": "+ThinkLess",
    "thinkbrake-prob": "+THINKBRAKE (prob)",
    "thinkbrake": "+THINKBRAKE",
    "oracle": "+Oracle",
}

df["model_display"] = df["model"].map(MODEL_DISPLAY_NAMES).fillna(df["model"])
print(f"Unique models: {df['model_display'].unique().tolist()}")
print(f"Unique methods: {df['method'].unique().tolist()}")
print(f"Unique benchmarks: {df['benchmark'].unique().tolist()}")

Unique models: ['Phi-4-reasoning-plus', 'Qwen3-4B', 'DeepSeek-R1-7B', 'Qwen3-4B-Thinking', 'Qwen3-32B', 'Qwen3-14B', 'Phi-4-reasoning']
Unique methods: ['rollout', 'thinkbrake', 'thinkbrake-prob', 'thinkless', 'nowait']
Unique benchmarks: ['bfcl-v1', 'meta-tool', 'bfcl-v2', 'gsm8k', 'aime2024', 'aime2025', 'math500', 'gpqa-diamond', 'arc-challenge']


## Table 1: Reasoning Benchmarks (Math + General)


In [4]:
REASONING_BENCHMARKS = [
    "gsm8k",
    "math500",
    "aime2024",
    "aime2025",
    "gpqa-diamond",
    "arc-challenge",
]

df_reasoning = df[
    (df["benchmark"].isin(REASONING_BENCHMARKS)) & (df["sub_category"].isna())
].copy()

print(f"Reasoning benchmark records: {len(df_reasoning)}")

Reasoning benchmark records: 320


In [5]:
def create_reasoning_table(df_data, model_order=None):
    benchmarks = [
        "gsm8k",
        "math500",
        "aime2024",
        "aime2025",
        "gpqa-diamond",
        "arc-challenge",
    ]

    if model_order is None:
        all_models = df_data["model"].unique()
    else:
        available_models = set(df_data["model"].unique())
        all_models = [m for m in model_order if m in available_models]

    rows = []

    for model in all_models:
        model_display = MODEL_DISPLAY_NAMES.get(model, model)
        model_data = df_data[df_data["model"] == model]
        model_threshold = MODEL_THRESHOLDS.get(model, DEFAULT_THRESHOLD)

        methods_to_show = [
            "rollout",
            "nowait",
            "thinkless",
            "thinkbrake-prob",
            "thinkbrake",
        ]

        baseline_data = {}
        rollout = model_data[model_data["method"] == "rollout"]
        for bench in benchmarks:
            bench_data = rollout[rollout["benchmark"] == bench]
            if len(bench_data) > 0:
                baseline_data[bench] = {
                    "accuracy": bench_data["accuracy"].values[0],
                    "tokens": bench_data["avg_token_length"].values[0],
                }

        first_row_for_model = True
        for method in methods_to_show:
            # Determine threshold to use
            if method == "thinkbrake":
                use_threshold = model_threshold
            elif method == "thinkbrake-prob":
                use_threshold = 0.25
            else:
                use_threshold = None

            # Filter by method and threshold
            if method in ["thinkbrake", "thinkbrake-prob"]:
                method_data = model_data[
                    (model_data["method"] == method)
                    & (model_data["threshold"] == use_threshold)
                ]
            else:
                method_data = model_data[model_data["method"] == method]

            if len(method_data) == 0:
                continue

            row = {
                "Model": model_display if first_row_for_model else "",
                "Method": METHOD_DISPLAY.get(method, method),
            }
            first_row_for_model = False

            acc_values = []

            for bench in benchmarks:
                bench_row = method_data[method_data["benchmark"] == bench]
                if len(bench_row) > 0:
                    acc = bench_row["accuracy"].values[0]
                    tokens = bench_row["avg_token_length"].values[0]

                    row[f"{bench}_acc"] = f"{acc:.1f}"
                    acc_values.append(acc)
                    row[f"{bench}_tok"] = f"{tokens:.0f}"
                else:
                    row[f"{bench}_acc"] = "–"
                    row[f"{bench}_tok"] = "–"

            # Calculate average accuracy
            if acc_values:
                avg_acc = np.mean(acc_values)
                row["avg_acc"] = f"{avg_acc:.1f}"

                # Calculate average token length
                if method != "rollout":
                    all_delta_tokens = []
                    for bench in benchmarks:
                        bench_row = method_data[method_data["benchmark"] == bench]
                        if len(bench_row) > 0 and bench in baseline_data:
                            tokens = bench_row["avg_token_length"].values[0]
                            all_delta_tokens.append(tokens)
                    if all_delta_tokens:
                        row["avg_tok"] = f"{np.mean(all_delta_tokens):.0f}"
                    else:
                        row["avg_tok"] = "–"
                else:
                    row["avg_tok"] = "–"
            else:
                row["avg_acc"] = "–"
                row["avg_tok"] = "–"

            rows.append(row)

    result_df = pd.DataFrame(rows)

    # Rename columns for display
    rename_map = {
        "gsm8k_acc": "GSM8K Acc",
        "gsm8k_tok": "Token",
        "math500_acc": "MATH500 Acc",
        "math500_tok": "Token",
        "aime2024_acc": "AIME24 Acc",
        "aime2024_tok": "Token",
        "aime2025_acc": "AIME25 Acc",
        "aime2025_tok": "Token",
        "gpqa-diamond_acc": "GPQA-D Acc",
        "gpqa-diamond_tok": "Token",
        "arc-challenge_acc": "ARC-C Acc",
        "arc-challenge_tok": "Token",
        "avg_acc": "Avg Acc",
        "avg_tok": "Token",
    }
    result_df = result_df.rename(columns=rename_map)

    return result_df


# Create the table with specified model order
reasoning_table = create_reasoning_table(df_reasoning, model_order=MODEL_ORDER)
display(Markdown("### Table 1: Reasoning Benchmarks"))
display(reasoning_table)

### Table 1: Reasoning Benchmarks

,Model,Method,GSM8K Acc,Token,MATH500 Acc,Token,AIME24 Acc,Token,AIME25 Acc,Token,GPQA-D Acc,Token,ARC-C Acc,Token,Avg Acc,Token
0,Qwen3-4B-Thinking,(baseline),95.1,1578,95.3,6360,76.7,19284,72.5,21317,64.1,8455,94.3,1178,83.0,–
1,,+NoWait,94.2,1182,96.4,5222,66.7,13404,66.7,13737,68.2,6796,92.7,1350,80.8,6948
2,,+ThinkLess,92.6,1498,56.4,3023,3.3,4230,3.3,4241,50.0,4165,94.2,1591,50.0,3125
3,,+THINKBRAKE (prob),93.9,990,91.2,2677,35.0,5277,30.4,5119,47.0,2749,93.7,805,65.2,2936
4,,+THINKBRAKE,94.7,1223,95.9,4760,63.7,12048,57.1,12455,57.6,5728,94.0,952,77.2,6194
5,Qwen3-4B,(baseline),94.4,2407,96.0,5250,68.8,13000,59.6,15163,51.0,7497,94.0,798,77.3,–
6,,+NoWait,94.5,1435,94.6,3602,56.7,10845,50.0,11058,57.6,5503,93.4,683,74.5,5521
7,,+ThinkLess,91.0,395,85.4,970,23.3,2685,23.3,2799,46.0,1182,89.3,116,59.7,1358
8,,+THINKBRAKE (prob),92.8,1439,88.6,2558,29.2,4617,25.0,4879,47.5,3435,93.4,678,62.7,2934
9,,+THINKBRAKE,94.4,1685,95.6,4419,66.7,11011,55.8,12696,50.0,5401,93.9,699,76.1,5985


---
## Table 2: Tool Benchmarks (BFCL + MetaTool)

BFCL sub-categories: parallel, parallel_multiple

In [6]:
df_tool = df[df["benchmark"].isin(["bfcl-v1", "bfcl-v2", "meta-tool"])].copy()
print(f"Tool benchmark records: {len(df_tool)}")
print(f"Sub-categories: {df_tool['sub_category'].dropna().unique().tolist()}")
print(f"Benchmarks: {df_tool['benchmark'].unique().tolist()}")

Tool benchmark records: 660
Sub-categories: ['simple', 'parallel', 'multiple', 'parallel_multiple', 'task2_subtask1', 'task2_subtask4']
Benchmarks: ['bfcl-v1', 'meta-tool', 'bfcl-v2']


In [7]:
def create_tool_table(
    df_data,
    model_order=None,
):
    bfcl_subcats = ["simple", "multiple", "parallel", "parallel_multiple"]
    metatool_subcats = ["task2_subtask1", "task2_subtask4"]
    metatool_display_names = {
        "task2_subtask1": "single",
        "task2_subtask4": "multiple",
    }

    if model_order is None:
        all_models = df_data["model"].unique()
    else:
        available_models = set(df_data["model"].unique())
        all_models = [m for m in model_order if m in available_models]

    rows = []

    for model in all_models:
        model_display = MODEL_DISPLAY_NAMES.get(model, model)
        model_data = df_data[df_data["model"] == model]
        model_threshold = MODEL_THRESHOLDS.get(model, DEFAULT_THRESHOLD)

        methods_to_show = [
            "rollout",
            "nowait",
            "thinkless",
            "thinkbrake-prob",
            "thinkbrake",
        ]

        baseline_data = {}
        rollout = model_data[model_data["method"] == "rollout"]

        for bench in ["bfcl-v1", "bfcl-v2"]:
            for subcat in bfcl_subcats:
                key = f"{bench}_{subcat}"
                data = rollout[
                    (rollout["benchmark"] == bench)
                    & (rollout["sub_category"] == subcat)
                ]
                if len(data) > 0:
                    baseline_data[key] = {
                        "accuracy": data["accuracy"].values[0],
                        "tokens": data["avg_token_length"].values[0],
                    }

        for subcat in metatool_subcats:
            key = f"meta-tool_{subcat}"
            data = rollout[
                (rollout["benchmark"] == "meta-tool")
                & (rollout["sub_category"] == subcat)
            ]
            if len(data) > 0:
                baseline_data[key] = {
                    "accuracy": data["accuracy"].values[0],
                    "tokens": data["avg_token_length"].values[0],
                }

        first_row_for_model = True
        for method in methods_to_show:
            if method == "thinkbrake":
                method_data = model_data[
                    (model_data["method"] == method)
                    & (model_data["threshold"] == model_threshold)
                ]
            elif method == "thinkbrake-prob":
                method_data = model_data[
                    (model_data["method"] == method)
                    & (model_data["threshold"] == DEFAULT_THRESHOLD)
                ]
            else:
                method_data = model_data[model_data["method"] == method]

            if len(method_data) == 0:
                continue

            row = {
                "Model": model_display if first_row_for_model else "",
                "Method": METHOD_DISPLAY.get(method, method),
            }
            first_row_for_model = False

            for bench in ["bfcl-v1", "bfcl-v2"]:
                for subcat in bfcl_subcats:
                    key = f"{bench}_{subcat}"
                    col_prefix = (
                        f"{bench.upper().replace('-', '')}_{subcat.replace('_','-')}"
                    )

                    data = method_data[
                        (method_data["benchmark"] == bench)
                        & (method_data["sub_category"] == subcat)
                    ]

                    if len(data) > 0:
                        acc = data["accuracy"].values[0]
                        tokens = data["avg_token_length"].values[0]

                        row[f"{col_prefix}_acc"] = f"{acc:.1f}"
                        row[f"{col_prefix}_tok"] = f"{tokens:.0f}"

                    else:
                        row[f"{col_prefix}_acc"] = "–"
                        row[f"{col_prefix}_tok"] = "–"

            # Add MetaTool sub-category columns
            for subcat in metatool_subcats:
                key = f"meta-tool_{subcat}"
                display_name = metatool_display_names.get(subcat, subcat)
                col_prefix = f"MT_{display_name}"

                data = method_data[
                    (method_data["benchmark"] == "meta-tool")
                    & (method_data["sub_category"] == subcat)
                ]

                if len(data) > 0:
                    acc = data["accuracy"].values[0]
                    tokens = data["avg_token_length"].values[0]
                    row[f"{col_prefix}_acc"] = f"{acc:.1f}"
                    row[f"{col_prefix}_tok"] = f"{tokens:.0f}"
                else:
                    row[f"{col_prefix}_acc"] = "–"
                    row[f"{col_prefix}_tok"] = "–"

            rows.append(row)

    result_df = pd.DataFrame(rows)
    return result_df


# Create the table with specified model order
tool_table = create_tool_table(df_tool, model_order=MODEL_ORDER)
display(Markdown("### Table 2: Tool Benchmarks (BFCL + MetaTool Sub-Categories)"))
display(tool_table)

### Table 2: Tool Benchmarks (BFCL + MetaTool Sub-Categories)

,Model,Method,BFCLV1_simple_acc,BFCLV1_simple_tok,BFCLV1_multiple_acc,BFCLV1_multiple_tok,BFCLV1_parallel_acc,BFCLV1_parallel_tok,BFCLV1_parallel-multiple_acc,BFCLV1_parallel-multiple_tok,BFCLV2_simple_acc,BFCLV2_simple_tok,BFCLV2_multiple_acc,BFCLV2_multiple_tok,BFCLV2_parallel_acc,BFCLV2_parallel_tok,BFCLV2_parallel-multiple_acc,BFCLV2_parallel-multiple_tok,MT_single_acc,MT_single_tok,MT_multiple_acc,MT_multiple_tok
0,Qwen3-4B-Thinking,(baseline),88.2,1072,90.5,965,90.5,1293,84.5,1727,79.8,1094,83.4,1832,87.5,1473,79.2,2397,69.7,1946,85.5,1336
1,,+NoWait,83.6,717,88.0,854,89.0,947,82.0,1231,75.6,767,80.3,1529,87.5,898,79.2,1680,68.9,1948,84.7,1186
2,,+ThinkLess,86.2,985,93.0,944,91.0,1177,83.5,1613,81.0,1030,82.9,1756,93.8,1321,79.2,2226,67.4,1820,86.5,1227
3,,+THINKBRAKE (prob),88.0,643,90.5,804,91.0,739,80.0,1027,79.5,713,81.7,1459,75.0,735,70.8,1588,70.8,1443,86.7,1166
4,,+THINKBRAKE,87.5,708,90.5,830,91.5,781,83.5,1089,81.0,776,83.7,1510,75.0,896,62.5,1610,72.4,1537,85.1,1206
5,Qwen3-4B,(baseline),88.2,607,92.0,744,90.0,844,83.0,1046,80.2,653,81.0,1426,81.2,837,70.8,1555,74.3,1069,90.5,856
6,,+NoWait,88.2,508,90.5,733,88.0,730,84.0,1059,78.3,590,80.0,1344,75.0,794,75.0,1382,72.5,1036,90.5,784
7,,+ThinkLess,86.4,297,89.5,509,87.5,388,83.5,620,71.3,363,76.4,1060,56.2,396,62.5,1003,69.1,757,92.2,536
8,,+THINKBRAKE (prob),87.3,525,93.0,710,90.5,719,84.0,996,80.2,608,81.8,1376,75.0,711,75.0,1357,73.5,1053,90.3,831
9,,+THINKBRAKE,86.9,520,92.0,717,87.5,680,84.0,931,79.1,596,80.4,1384,75.0,656,75.0,1429,72.7,1062,89.3,826


---
## Table 3: Extended Metrics (pass@5, majority@8, avg@8)

AIME와 MATH500에 대한 추가 메트릭을 표시합니다.

In [8]:
def create_extended_metrics_table(
    df_data, main_model="Qwen_Qwen3-4B-Thinking-2507", model_order=None
):
    """
    Create a table showing pass@5, majority@8, avg@8 for AIME and MATH500.
    Uses model-specific thresholds from MODEL_THRESHOLDS.
    """
    benchmarks = ["math500", "aime2024", "aime2025"]

    # Filter for extended metrics benchmarks (should have pass@k, majority_accuracy, avg@8)
    df_ext = df_data[
        (df_data["benchmark"].isin(benchmarks)) & (df_data["sub_category"].isna())
    ].copy()

    if model_order is None:
        all_models = df_ext["model"].unique()
    else:
        available_models = set(df_ext["model"].unique())
        all_models = [m for m in model_order if m in available_models]

    rows = []

    for model in all_models:
        model_display = MODEL_DISPLAY_NAMES.get(model, model)
        model_data = df_ext[df_ext["model"] == model]
        threshold = MODEL_THRESHOLDS.get(model, DEFAULT_THRESHOLD)

        # Determine methods to show
        if model == main_model:
            methods_to_show = ["rollout", "thinkbrake"]
        else:
            methods_to_show = ["rollout", "thinkbrake"]

        first_row_for_model = True
        for method in methods_to_show:
            if method in ["thinkbrake", "thinkbrake-prob"]:
                method_data = model_data[
                    (model_data["method"] == method)
                    & (model_data["threshold"] == threshold)
                ]
            else:
                method_data = model_data[model_data["method"] == method]

            if len(method_data) == 0:
                continue

            row = {
                "Model": model_display if first_row_for_model else "",
                "Method": METHOD_DISPLAY.get(method, method),
            }
            first_row_for_model = False

            for bench in benchmarks:
                bench_row = method_data[method_data["benchmark"] == bench]
                if len(bench_row) > 0:
                    # pass@k is a dict like {"1": 68.75, "5": 82.55}
                    pass_k = bench_row["pass@k"].values[0]
                    if isinstance(pass_k, dict) and "5" in pass_k:
                        row[f"{bench}_pass5"] = f"{pass_k['5']:.1f}"
                    else:
                        row[f"{bench}_pass5"] = "–"

                    # majority_accuracy
                    maj = bench_row.get("majority_accuracy")
                    if maj is not None and len(maj) > 0 and pd.notna(maj.values[0]):
                        row[f"{bench}_maj8"] = f"{maj.values[0]:.1f}"
                    else:
                        row[f"{bench}_maj8"] = "–"

                    # avg@8
                    avg8 = bench_row.get("avg@8")
                    if avg8 is not None and len(avg8) > 0 and pd.notna(avg8.values[0]):
                        row[f"{bench}_avg8"] = f"{avg8.values[0]:.1f}"
                    else:
                        row[f"{bench}_avg8"] = "–"
                else:
                    row[f"{bench}_pass5"] = "–"
                    row[f"{bench}_maj8"] = "–"
                    row[f"{bench}_avg8"] = "–"

            rows.append(row)

    result_df = pd.DataFrame(rows)

    # Rename columns
    rename_map = {
        "math500_pass5": "MATH500 pass@5",
        "math500_maj8": "maj@8",
        "aime2024_pass5": "AIME24 pass@5",
        "aime2024_maj8": "maj@8",
        "aime2025_pass5": "AIME25 pass@5",
        "aime2025_maj8": "maj@8",
    }
    result_df = result_df.rename(columns=rename_map)

    return result_df


# Create extended metrics table
extended_table = create_extended_metrics_table(df, model_order=MODEL_ORDER)
display(Markdown("### Table 3: Extended Metrics (pass@5, majority@8, avg@8)"))
display(extended_table)

### Table 3: Extended Metrics (pass@5, majority@8, avg@8)

,Model,Method,MATH500 pass@5,maj@8,math500_avg8,AIME24 pass@5,maj@8,aime2024_avg8,AIME25 pass@5,maj@8,aime2025_avg8
0,Qwen3-4B-Thinking,(baseline),98.3,97.2,95.3,86.2,86.7,76.7,84.1,80.0,72.5
1,,+THINKBRAKE,98.7,97.2,95.9,81.2,73.3,63.7,76.1,70.0,57.1
2,Qwen3-4B,(baseline),98.7,97.2,96.0,82.6,80.0,68.8,77.0,73.3,59.6
3,,+THINKBRAKE,98.4,97.4,95.6,78.6,76.7,66.7,73.9,70.0,55.8
4,Qwen3-14B,(baseline),99.2,97.8,96.8,82.1,80.0,71.7,79.6,76.7,69.2
5,,+THINKBRAKE,98.9,98.0,96.9,83.0,80.0,75.8,82.5,73.3,67.9
6,DeepSeek-R1-7B,(baseline),97.9,96.6,93.8,76.1,76.7,54.2,61.4,50.0,38.8
7,,+THINKBRAKE,97.2,95.0,91.5,74.1,70.0,48.3,51.2,46.7,35.4
8,Qwen3-32B,(baseline),99.2,98.2,97.0,88.6,86.7,75.8,87.1,80.0,68.3
9,,+THINKBRAKE,99.4,98.8,97.5,89.3,86.7,78.3,83.4,76.7,63.7


## Threshold Analysis for ThinkBrake

In [9]:
PARENT_CATEGORIES = {
    "gsm8k": "math",
    "math500": "math",
    "aime2024": "math",
    "aime2025": "math",
    "gpqa-diamond": "general",
    "arc-challenge": "general",
    "bfcl-v1": "tool",
    "bfcl-v2": "tool",
    "meta-tool": "tool",
}

df_tb = df[(df["method"] == "thinkbrake") & (df["sub_category"].isna())].copy()
df_tb["category"] = df_tb["benchmark"].map(PARENT_CATEGORIES)

threshold_analysis = df_tb.pivot_table(
    index=["model", "threshold"], columns="category", values="accuracy", aggfunc="mean"
)

print("ThinkBrake Threshold Analysis by Category (Average Accuracy)")
display(threshold_analysis.style.format("{:.2f}").background_gradient(axis=0))

ThinkBrake Threshold Analysis by Category (Average Accuracy)
